### FacebookAI/roberta-base
https://huggingface.co/FacebookAI/roberta-base

### No. 6

In [ ]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset, DatasetDict
import torch
from transformers import get_linear_schedule_with_warmup

os.environ["TRANSFORMERS_VERBOSITY"] = "error"
os.environ["WANDB_DISABLED"] = "true"

# Load data
train_df = pd.read_csv('LT-EDI-ACL2022_Dataset/train.csv')
dev_df = pd.read_csv('LT-EDI-ACL2022_Dataset/dev.tsv', sep='\t')
test_df = pd.read_csv('LT-EDI-ACL2022_Dataset/test.csv')

# Drop unnecessary columns
train_df.drop("pid", axis=1, inplace=True)
dev_df.drop("PID", axis=1, inplace=True)
test_df.drop("pid", axis=1, inplace=True)

# Rename columns for dev_df
dev_df = dev_df.rename(columns={'Text data': 'text', 'Label': 'labels'})

# Map label names to integers
label_mapping = {'severe': 0, 'moderate': 1, 'not depression': 2}
dev_df['labels'] = dev_df['labels'].map(label_mapping)

# Convert DataFrame to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(dev_df)
test_dataset = Dataset.from_pandas(test_df)

# Merge datasets into a DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Tokenizer: Load from FacebookAI/roberta-base
model_checkpoint = "roberta-base"  # Changed to FacebookAI's base model
tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)

# Tokenize the dataset
def preprocess_function(examples):
    cleaned_texts = [str(text) if text else "" for text in examples["text"]]
    return tokenizer(cleaned_texts, truncation=True, max_length=128)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Load the base model (Roberta base) for sequence classification with 3 classes
model = RobertaForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3, ignore_mismatched_sizes=True)

# Data Collator: Use padding
data_collator = DataCollatorWithPadding(tokenizer)

# Define evaluation metrics (accuracy)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = torch.argmax(torch.tensor(predictions), dim=-1)
    accuracy = (preds == torch.tensor(labels)).float().mean().item()
    return {"accuracy": accuracy}

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results_6",  # Updated output directory
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs_6',  # Updated logging directory
    logging_steps=10,
)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

# Scheduler for learning rate
num_training_steps = len(tokenized_datasets["train"]) // training_args.per_device_train_batch_size * training_args.num_train_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Trainer class
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    optimizers=(optimizer, scheduler),
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
print("Validation metrics:")
trainer.evaluate(eval_dataset=tokenized_datasets["validation"])

print("Test metrics:")
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

# Save the trained model and tokenizer
model.save_pretrained('./trained_model_6')
tokenizer.save_pretrained('./trained_model_6')

# Get predictions for the test set using the trained model
def get_predictions_and_logits(trainer, dataset):
    predictions, labels, metrics = trainer.predict(dataset)
    
    # Convert logits to predicted labels 
    predicted_labels = np.argmax(predictions, axis=-1)

    return predictions, predicted_labels, labels

# Get predictions, logits, and true labels for the test dataset
predictions, predicted_labels, true_labels = get_predictions_and_logits(trainer, tokenized_datasets["test"])

# Convert logits to pandas DataFrame for easy manipulation
logits_df = pd.DataFrame(predictions, columns=["logit_" + str(i) for i in range(predictions.shape[1])])

# Create a DataFrame with text, true labels, and predicted labels
results_df = pd.DataFrame({
    'text': tokenized_datasets["test"]["text"],
    'label': true_labels,
    'label_pred': predicted_labels
})

# Concatenate logits DataFrame with the results_df
final_df = pd.concat([results_df, logits_df], axis=1)
final_df.to_csv("test_predictions_with_logits_6.csv", index=False)

# Filter out misclassified samples
misclassified_df = results_df[results_df['label'] != results_df['label_pred']]
misclassified_df.to_csv('misclassified_samples_6.csv', index=False)

# Count per label
misclassified_counts = misclassified_df['label'].value_counts()

# Display distribution of misclassified labels
print(f"Misclassification distribution:\n{misclassified_counts}")

### No. 8

In [ ]:
# Load data
train_df = pd.read_csv('Preprocessed_Dataset/train.csv')
val_df = pd.read_csv('Preprocessed_Dataset/dev.csv')
test_df = pd.read_csv('Preprocessed_Dataset/test.csv')

# Convert DataFrame to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Merge datasets
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Tokenizer: Load from last training
model_checkpoint = RobertaForSequenceClassification.from_pretrained('./trained_model_6')
tokenizer = RobertaTokenizer.from_pretrained('./trained_model_6')

# Tokenize
def preprocess_function(examples):
    cleaned_texts = [str(text) if text else "" for text in examples["text"]]
    return tokenizer(cleaned_texts, truncation=True, max_length=128)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Load the model
model = RobertaForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, ignore_mismatched_sizes=True)  # 2 classes

# Data Collector
data_collator = DataCollatorWithPadding(tokenizer)

# Evaluation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = torch.argmax(torch.tensor(predictions), dim=-1)
    accuracy = (preds == torch.tensor(labels)).float().mean().item()
    return {"accuracy": accuracy}

# parameters
training_args = TrainingArguments(
    output_dir="./results_8",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs_8',
    logging_steps=10,
)

# optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

# learning rate scheduler
num_training_steps = len(tokenized_datasets["train"]) // training_args.per_device_train_batch_size * training_args.num_train_epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=num_training_steps)

# Trainer class
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    optimizers=(optimizer, scheduler),
    compute_metrics=compute_metrics,
)

# Train
trainer.train()

# Evaluate
print("Validation metrics:")
trainer.evaluate(eval_dataset=tokenized_datasets["validation"])

print("Test metrics:")
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

# Save
model.save_pretrained('./trained_model_8')
tokenizer.save_pretrained('./trained_model_8')

# Get predictions for the test set using the trained model
def get_predictions_and_logits(trainer, dataset):
    predictions, labels, metrics = trainer.predict(dataset)
    
    # Convert logits to predicted labels (0 or 1)
    predicted_labels = np.argmax(predictions, axis=-1)

    return predictions, predicted_labels, labels

# Get predictions, logits, and true labels for the test dataset
predictions, predicted_labels, true_labels = get_predictions_and_logits(trainer, tokenized_datasets["test"])

# Convert logits to pandas DataFrame for easy manipulation
logits_df = pd.DataFrame(predictions, columns=["logit_" + str(i) for i in range(predictions.shape[1])])

# Create a DataFrame with text, true labels, and predicted labels
results_df = pd.DataFrame({
    'text': tokenized_datasets["test"]["text"],
    'label': true_labels,
    'label_pred': predicted_labels
})

# Concatenate logits DataFrame with the results_df
final_df = pd.concat([results_df, logits_df], axis=1)
final_df.to_csv("test_predictions_with_logits_8.csv", index=False)

# Filter out misclassified samples
misclassified_df = results_df[results_df['label'] != results_df['label_pred']]
misclassified_df.to_csv('misclassified_samples_8.csv', index=False)

# Count per label
misclassified_counts = misclassified_df['label'].value_counts()

# Display distribution
print(f"Misclassification distribution:\n{misclassified_counts}")